In [1]:
from IPython.core.display import HTML

with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Chess core functions

### Imports

In [2]:
import chess
import chess.polyglot
import math
import random
import heapq

## Auxiliary functions

### Hash function

In [3]:
def zobrist_hash(board):
    return chess.polyglot.zobrist_hash(board)

### Piece-Square Tables
Values are taken from: https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece-Square_Tables  
These tables represent the values for the black pieces.  
For the king, two different tables are used, depending on wether the board is in the endgame or not (for further explanation, see below).

In [4]:
PAWN_VALUES = [0, 0, 0, 0, 0, 0, 0, 0,
               50, 50, 50, 50, 50, 50, 50, 50,
               10, 10, 20, 30, 30, 20, 10, 10,
               5, 5, 10, 25, 25, 10, 5, 5,
               0, 0, 0, 20, 20, 0, 0, 0,
               5, -5, -10, 0, 0, -10, -5, 5,
               5, 10, 10, -20, -20, 10, 10, 5,
               0, 0, 0, 0, 0, 0, 0, 0]

KNIGHT_VALUES = [-50, -40, -30, -30, -30, -30, -40, -50,
                 -40, -20, 0, 0, 0, 0, -20, -40,
                 -30, 0, 10, 15, 15, 10, 0, -30,
                 -30, 5, 15, 20, 20, 15, 5, -30,
                 -30, 0, 15, 20, 20, 15, 0, -30,
                 -30, 5, 10, 15, 15, 10, 5, -30,
                 -40, -20, 0, 5, 5, 0, -20, -40,
                 -50, -40, -30, -30, -30, -30, -40, -50]

BISHOP_VALUES = [-20, -10, -10, -10, -10, -10, -10, -20,
                 -10, 0, 0, 0, 0, 0, 0, -10,
                 -10, 0, 5, 10, 10, 5, 0, -10,
                 -10, 5, 5, 10, 10, 5, 5, -10,
                 -10, 0, 10, 10, 10, 10, 0, -10,
                 -10, 10, 10, 10, 10, 10, 10, -10,
                 -10, 5, 0, 0, 0, 0, 5, -10,
                 -20, -10, -10, -10, -10, -10, -10, -20]

ROOK_VALUES = [0, 0, 0, 0, 0, 0, 0, 0,
               5, 10, 10, 10, 10, 10, 10, 5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               0, 0, 0, 5, 5, 0, 0, 0]

QUEEN_VALUES = [-20, -10, -10, -5, -5, -10, -10, -20,
                -10, 0, 0, 0, 0, 0, 0, -10,
                -10, 0, 5, 5, 5, 5, 0, -10,
                -5, 0, 5, 5, 5, 5, 0, -5,
                0, 0, 5, 5, 5, 5, 0, -5,
                -10, 5, 5, 5, 5, 5, 0, -10,
                -10, 0, 5, 0, 0, 0, 0, -10,
                -20, -10, -10, -5, -5, -10, -10, -20]

KING_VALUES_MIDDLEGAME = [-30, -40, -40, -50, -50, -40, -40, -30,
                          -30, -40, -40, -50, -50, -40, -40, -30,
                          -30, -40, -40, -50, -50, -40, -40, -30,
                          -30, -40, -40, -50, -50, -40, -40, -30,
                          -20, -30, -30, -40, -40, -30, -30, -20,
                          -10, -20, -20, -20, -20, -20, -20, -10,
                          20, 20, 0, 0, 0, 0, 20, 20,
                          20, 30, 10, 0, 0, 10, 30, 20]

KING_VALUES_ENDGAME = [-50, -40, -30, -20, -20, -30, -40, -50,
                       -30, -20, -10, 0, 0, -10, -20, -30,
                       -30, -10, 20, 30, 30, 20, -10, -30,
                       -30, -10, 30, 40, 40, 30, -10, -30,
                       -30, -10, 30, 40, 40, 30, -10, -30,
                       -30, -10, 20, 30, 30, 20, -10, -30,
                       -30, -30, 0, 0, 0, 0, -30, -30,
                       -50, -30, -30, -30, -30, -30, -30, -50]


### Checking for endgame

The function `is_endgame` takes the current `board` and returns with help of `color_endgame` wether both players match one of the two criteria for the endgame according to the simple evaluation function. This means, the endgame beings if:
1. Both sides have no queens or
2. Every side which has a queen has additionally no other pieces or one minorpiece maximum

(See https://www.chessprogramming.org/Simplified_Evaluation_Function#King).

In [5]:
def is_endgame(board):
    white_endgame = color_endgame(board, chess.WHITE)
    black_endgame = color_endgame(board, chess.BLACK)
    return white_endgame and black_endgame


def color_endgame(board, color):
    return (len(board.pieces(chess.QUEEN, color)) == 0 or
            len(board.pieces(chess.BISHOP, color)) + len(board.pieces(chess.KNIGHT, color)) < 2 and
            len(board.pieces(chess.ROOK, color)) == 0)

### Function for the Piece-Square Tables

This function calculates the value using the `piece_values` (according to the piece-square tables), `player_pieces` and `enemy_pieces` (the list of the player's respectively the opponent's pieces) as well as the current `board` and returns the calculated value.

In [6]:
def piece_square_tables(piece_square_values, player_pieces, enemy_pieces, board):
    value = 0
    black_piece_square_values = piece_square_values
    white_piece_square_values = piece_square_values[::-1]
    for piece in player_pieces:
        if board.turn:
            value += white_piece_square_values[piece]
        else:
            value += black_piece_square_values[piece]
    for piece in enemy_pieces:
        if board.turn:
            value -= black_piece_square_values[piece]
        else:
            value -= white_piece_square_values[piece]
    return value

### Maximum value (32-bit integer)

In [7]:
WIN_VALUE = 2147483647

## Static evaluation

The function `static_eval` takes the parameters `board` (the current board) and `endgame` (if the players are in the endgame) and evaluates the current state of the board. First, the function checks, if the game is over and returns the corresponsing value (e.g. if it is white's turn and white won, it returns the maximum value defined above). If the game is not over, the function computes the total values for each piece type of both sides by multiplying the piece value* with the amount of pieces on the board. Lastly, the function uses the auxiliary function `piece_square_tables` to calculate the values given by the positions of the pieces.

*The piece values are:
- Pawn = 100 cp
- Knight = 320 cp
- Bishop = 330 cp
- Rook = 500 cp
- Queen = 900 cp

`cp` stands for *centipawn* i.e. 0.01 pawn.

In [8]:
def static_eval(board, endgame):
    # STEP ONE: Check, if the game is over
    # White won
    if (board.result() == "1-0" and board.turn or
            board.result() == "0-1" and not board.turn):
        result = WIN_VALUE - board.fullmove_number
        return result
    # Black won
    elif (board.result() == "0-1" and board.turn or
          board.result() == "1-0" and not board.turn):
        result = -WIN_VALUE + board.fullmove_number
        return result
    # Draw
    elif (board.result() == "1/2-1/2" or
          board.is_stalemate() or
          board.is_insufficient_material() or
          board.is_seventyfive_moves() or
          board.is_fivefold_repetition()):
        result = 0
        return result
    # STEP TWO: Calculate piece values using the piece-square tables
    # Value that will be returned eventually
    value = 0
    # Piece values in centipawns as explained above
    piece_values = [100, 320, 330, 500, 900, 10000]
    for piece_type, piece_value in enumerate(piece_values):
        piece_type += 1
        player_pieces = board.pieces(piece_type, board.turn)
        enemy_pieces = board.pieces(piece_type, not board.turn)
        value += len(player_pieces) * piece_value
        value -= len(enemy_pieces) * piece_value
        if piece_type == chess.PAWN:
            value += piece_square_tables(PAWN_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.KNIGHT:
            value += piece_square_tables(KNIGHT_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.BISHOP:
            value += piece_square_tables(BISHOP_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.ROOK:
            value += piece_square_tables(ROOK_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.QUEEN:
            value += piece_square_tables(QUEEN_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.KING:
            if endgame:
                value += piece_square_tables(KING_VALUES_ENDGAME, player_pieces, enemy_pieces, board)
            else:
                value += piece_square_tables(KING_VALUES_MIDDLEGAME, player_pieces, enemy_pieces, board)
    return value

## Minimax function

The main function, implementing the minimax algorithm (technically Negamax), takes the following parameters:
- `board`, the current board
- `depth`, the depth to which the board will be analyzed
- `alpha` and `beta`, needed for Alpha-Beta-Pruning

The function returns the biggest calculated value and it sets the global variable `BEST_MOVE` which is the move to play.  
The function also uses a transposition table (here called `cache`) to save computational ressources. It uses the `zobrist_hash` function to create a cache key made out of the hash as well as the current depth. Inside the cache, the following parameters are being saved:
- the calculated `value`
- the best `move`
- `alpha`
- `beta`

First, the function checks, if the key, i.e. the current board and the current depth, are already in the cache. If so, 

In [9]:
ANALYZING_DEPTH = None


def memoize_minimax(f):
    global CACHE
    global BEST_MOVE
    global CACHE_HITS

    def f_memoized(*args):
        global CACHE
        global BEST_MOVE
        global CACHE_HITS
        board, depth, alpha, beta = args[0], args[1], args[2], args[3]
        key = (zobrist_hash(board), depth)
        if key in CACHE.keys():
            CACHE_HITS += 1
            c_value, c_move, c_alpha, c_beta = CACHE[key]
            if c_alpha <= alpha and beta <= c_beta:
                if depth == ANALYZING_DEPTH:
                    BEST_MOVE = c_move
                return c_value
        result = f(*args)
        CACHE[key] = result, BEST_MOVE, alpha, beta
        return result

    return f_memoized


@memoize_minimax
def minimax(board, depth, alpha, beta):
    global BEST_MOVE
    global MINIMAX_CALLS
    MINIMAX_CALLS += 1
    # Check, if current board is in cache
    # If the depth is zero, give the static evaluation of the current board and save it in the cache
    if depth == 0 or not board.legal_moves:
        value = static_eval(board, is_endgame(board))
        return value
    max_value = alpha
    ordered_moves = []
    cnt = 0
    # Order the moves roughly, using a static evaluation for every move
    for move in board.legal_moves:
        cnt += 1
        board.push(move)
        v = static_eval(board, is_endgame(board))
        board.pop()
        heapq.heappush(ordered_moves, (v, cnt, move))
    # Calculate the minimax value recursively, using alpha-beta-pruning
    for _, _, move in ordered_moves:
        board.push(move)
        value = -minimax(board, depth - 1, -beta, -max_value)
        board.pop()
        if value > max_value:
            max_value = value
            if depth == ANALYZING_DEPTH:
                BEST_MOVE = move
            if max_value >= beta:
                break
    return max_value

### Input function for minimax
This function is the first to call; it sets the `cache`, initialises some debug/logging parameters and checks, if the move is covered by the opening library (`get_polyglot_move`). The function takes the `board` and the desired `depth`.

In [10]:
BEST_MOVE = None
CACHE = {}
CACHE_HITS = 0
MINIMAX_CALLS = 0


def minimax_input(board, depth):
    global CACHE
    CACHE = {}
    global MINIMAX_CALLS
    MINIMAX_CALLS = 0
    global CACHE_HITS
    CACHE_HITS = 0
    # DEBUG/LOGGING
    global ANALYZING_DEPTH
    ANALYZING_DEPTH = depth
    value, polyglot_move = get_polyglot_move(board)
    if polyglot_move:
        return value, polyglot_move
    return minimax(board, depth, -math.inf, math.inf), BEST_MOVE

### Using the opening library
This auxiliary function takes the current `board` and checks if the played move is in an external opening library. It then chooses a random move, proposed by the library and returns the value 0 and the `move`.

In [11]:
POLYGLOT_PATH = "data/polyglot/performance.bin"


def get_polyglot_move(board):
    with chess.polyglot.open_reader(POLYGLOT_PATH) as reader:
        l = list(reader.find_all(board))
        if l:
            return 0, random.choice(l).move
        return 0, None